In [1]:
import pandas as pd
import numpy as np
import sqlite3

In [2]:
trips = pd.read_csv('../../CSVs/trips.csv', low_memory=False)
stop_times = pd.read_csv('../../CSVs/stop_times.csv', low_memory=False)
#st_update = pd.read_csv('../../CSVs/stop_time_updates.csv', usecols=['stop_sequence', 'arrival_time', 'trip_update_id'], low_memory=False)
trp_update = pd.read_csv('../../CSVs/trip_updates.csv', usecols=['trip_id', 'route_id', 'timestamp', 'trip_start_time'], low_memory=False)

In [3]:
trp_update.head()

,trip_id,route_id,trip_start_time,timestamp
0,35242107,1,NaN,2017-09-25 22:33:24.000000
1,35242108,1,NaN,2017-09-25 22:33:24.000000
2,35242109,1,NaN,2017-09-25 22:33:24.000000
3,35242110,1,NaN,2017-09-25 22:33:24.000000
4,35242111,1,NaN,2017-09-25 22:33:24.000000


In [4]:
stop_times = stop_times.loc[:, ['trip_id', 'stop_id', 'stop_sequence', 'arrival_time', 'departure_time']]
stop_times.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
1,Logan-22-Weekday-trip,Logan-RentalCarCenter,2,08:04:00,08:04:00
2,Logan-22-Weekday-trip,Logan-A,3,08:09:00,08:09:00
3,Logan-22-Weekday-trip,Logan-B,4,08:12:00,08:12:00
4,Logan-22-Weekday-trip,Logan-Subway,5,08:17:00,08:17:00


In [5]:
trips = trips.loc[:, ['trip_id', 'route_id', 'service_id', 'trip_headsign']]
trips.head()

,trip_id,route_id,service_id,trip_headsign
0,Logan-22-Weekday-trip,Logan-22,Logan-Weekday,Loop
1,Logan-22-Weekend-trip,Logan-22,Logan-Weekend,Loop
2,Logan-33-Weekday-trip,Logan-33,Logan-Weekday,Loop
3,Logan-33-Weekend-trip,Logan-33,Logan-Weekend,Loop
4,Logan-55-Weekday-trip,Logan-55,Logan-Weekday,Loop


Obtain all the first stops for all trips as well as their expected arrival time.

In [6]:
first_stops = stop_times.loc[stop_times['stop_sequence'] == 1, :]
first_stops.head()

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
0,Logan-22-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
7,Logan-22-Weekend-trip,Logan-Subway,1,12:00:00,12:00:00
14,Logan-33-Weekday-trip,Logan-Subway,1,08:00:00,08:00:00
21,Logan-33-Weekend-trip,Logan-Subway,1,12:00:00,12:00:00
28,Logan-55-Weekday-trip,Logan-Subway,1,00:00:00,00:00:00


In [14]:
[trp_update['timestamp'].min(),trp_update['timestamp'].max()]

['2017-09-25 22:33:24.000000', '2017-09-26 12:33:39.000000']

Above is the time the first update and last update were obtained. Now to find trips that are running around the same time real time data was being gathered.

In [16]:
first_stops[first_stops['arrival_time'] == '21:00:00'][:25]

,trip_id,stop_id,stop_sequence,arrival_time,departure_time
1662,CR-Saturday-October-17-L-1314,Lowell,1,21:00:00,21:00:00
1790,CR-Saturday-August-17-L-1314,Lowell,1,21:00:00,21:00:00
3591,CR-Saturday-Spring-17-1160,Newburyport,1,21:00:00,21:00:00
4691,CR-Sunday-October-17-L-2314,Lowell,1,21:00:00,21:00:00
4819,CR-Sunday-August-17-L-2314,Lowell,1,21:00:00,21:00:00
6313,CR-Sunday-Spring-17-2160,Newburyport,1,21:00:00,21:00:00
8845,CR-Weekday-Spring-17-530,Worcester,1,21:00:00,21:00:00
9852,CR-Weekday-Spring-17-727,South Station,1,21:00:00,21:00:00
10267,CR-Weekday-Spring-17-784,Readville,1,21:00:00,21:00:00
10275,CR-Weekday-Spring-17-785,South Station,1,21:00:00,21:00:00


In [8]:
# connect to the sqlite database for querying data from stop time updates
conn = sqlite3.connect('../../gtfs.db')